# Queing Theory

## Littles laws

* L = expected number of costumers in the system
* L_q = expected queue length
* W = waiting time in system
* W_q = waiting time in queue
* lambda = constant arrival rate with mean 1/lambda
* mu = service rate with mean 1/mu

* L = lambda * W 
* L_q = lambda W_q
* W = W_q + 1/mu

### Birth and Death process

Assumptions
* Given N(t) = n the probability distribution of the remaining time until the next arrival is exponential with paramater lambda_n
* Given N(t) = n the probability distribution of the remaining time until the next service completion is exponential with paramater mu_n
* The random variables for remaining time until these are mutually independent. The next transition is either a single arrival or a single service completion

### Stationary Behaviour

Rate in = Rate out

In [ ]:
# # # System dependent
# # Pn = (lambda_n1*lambda_n2*lambda_n0)/(mu_n1*mu_n2*mu_n0)*P0
# # L = sum(n*Pn, n = 0:infinity)
# # Lq = sum((n-s)*Pn, n = s:infinity)
# # W = L/mean(lambda)
# # Wq = Lq/mean(lambda)

# # # For constant with a single server
# # P0 = 1-lambda/mu
# # Pn = (lambda/mu)^n*P0
# # L = lambda/(mu-lambda)
# # W = 1/(mu-lambda)
# # Wq = 1/(mu*(mu-lambda))
# # Lq = lambda^2/(mu*(mu-lambda))

In [98]:
# Not fully tested, use with caution
function Que_dependent(lambda::Vector{Float64}, mu::Vector{Float64}, s::Int64)
    # # # System dependent
    Pn = []
    Po = 0
    L = 0
    Lq = 0

    # Finding P0
    for n in 1:length(lambda)
        Po += prod(lambda[1:n])/prod(mu[2:n])
    end
    Po = 1/Po

    # Finding Pn
    for n in 1:length(lambda)
        push!(Pn,prod(lambda[1:n])/prod(mu[2:n])*Po)
    end

    # Finding L
    for n in 1:length(Pn)
        L += n*Pn[n]
    end

    # Finding Lq
    for n in s:length(Pn)
        Lq += (n-s)*Pn[n]
    end
    
    # Finding W
    W = L/mean(lambda)

    # Finding Wq
    Wq = Lq/mean(lambda)

    return Pn, L, Lq, W, Wq
end
Que_dependent(float([1,1,1]),float([0,2,2]),1)


(Any[0.5714285714285714, 0.2857142857142857, 0.14285714285714285], 1.5714285714285714, 0.5714285714285714, 1.5714285714285714, 0.5714285714285714)

In [114]:

function Que_stationary(lambda, mu, n = 1)
    P0 = 1-lambda/mu
    Pn = (lambda/mu)^n*P0
    L = lambda/(mu-lambda)
    W = 1/(mu-lambda)
    Wq = 1/(mu*(mu-lambda))
    Lq = lambda^2/(mu*(mu-lambda))
    return P0,Pn,L,W,Wq,Lq
end

Que_stationary(10,15,1)

(0.33333333333333337, 0.22222222222222224, 2.0, 0.2, 0.013333333333333334, 1.3333333333333333)

## M/M/s

### Functions

In [17]:

function P0(lambda,mu,s)
    a = 0
    for n in 0:s-1 
        a += (((lambda/mu)^n)/factorial(n))
    end
    a += ((lambda/mu)^s)/factorial(s)*1/(1-lambda/(s*mu))
    a = 1/a
    return a
end

# Stationary probabilities
function Que_stationary_MMs(lambda, mu, s)
    rho = lambda/(s*mu) # er det rigtigt ? 
    Lq = P0(lambda,mu,s)*(lambda/mu)^s*rho/(factorial(s)*(1-rho)^2)
    Wq = Lq/lambda
    W = Wq+1/mu
    # Stationary probabilities L for a M/M/s queue system by littles law
    L = lambda*(W)
    L = Lq+lambda/mu
    return L,Lq,W,Wq
end

Que_stationary_MMs(13.87,1/(5/60),3)



(1.2367569924347888, 0.08092365910145545, 0.0891677716247144, 0.005834438291381071)

In [16]:
function Cn(n,lambda,mu,s)
    if n <= s
        Cn = ((lambda/mu)^n)/factorial(n)
    else
        Cn = ((lambda/mu)^n)/(s!*s^(n-s))
    end
end

function P0(lambda,mu,s)
    a = 0
    for n in 0:s-1 
        a += (((lambda/mu)^n)/factorial(n))
    end
    a += ((lambda/mu)^s)/factorial(s)*1/(1-lambda/(s*mu))
    a = 1/a
    return a
end

function Pn(n,lambda,mu,s)
    if lambda <= s*mu
        po = P0(lambda,mu,s)
    end

    if n <= s
        pn = (lambda/mu)^n/factorial(n)*po
    else
        pn = (lambda/mu)^n/(factorial(s)*s^(n-s))
    end
    return pn
end

function MMs(n,lambda,mu,s)
    pn = 0
    cn = 0
    p0 = 0
    if lambda < s*mu
        p0 = P0(lambda,mu,s)
        pn = Pn(n,lambda,mu,s)
    else
        println("Might be wrong. Lambda is smaller than s*mu")
    end
    return pn,p0
end

# Ignore the first output value
# The second is Pn and the third is P0
MMs(3.86,4,6,3)


(0.013546035536748737, 0.5121951219512195)

## Terms

### Queu

THere is an arrival process and a service process

### Balking

lambda_n: customer refusing to enter the system

### Renege

mu_n: customer enters the system, but leaves without being served

## Distributions

Poisson: lambda^k*exp(-lambda)/k!

## Exercizes

In [ ]:
# 17.4.1
using Distributions
using Statistics

lambda = 2  # husk hvad den tager imod i julia
mu = 2

# a
cdf(Exponential(lambda), 1) # (i)
cdf(Exponential(lambda), 2)-cdf(Exponential(lambda), 1) # (ii)
1-cdf(Exponential(lambda), 2) # (iii)

# b
cdf(Exponential(mu), 1)

# c
# Poisson uses 1/lambda as the parameter where exponential uses lambda
pdf(Poisson(1/lambda), 0) # (i)
pdf(Poisson(1/lambda), 1) # (ii)
1-cdf(Poisson(1/lambda), 1) # (iii)

# d

# 17.4.7
# For exponential distribution mean and std are the same
# two exponential distributions summed up gives an exponential distribution
# therefor since we have service time and two of them we can half the expected waiting time
# 5 minuttes

#17.5.2
